In [1]:
import networkx as nx
import itertools
import math

# List of the lower 48 states with their major airports (ICAO codes and coordinates)
states_airports = {
    'Alabama': ('KBHM', (33.5629, -86.7535)),
    'Arizona': ('KPHX', (33.4342, -112.0116)),
    'Arkansas': ('KLIT', (34.7294, -92.2243)),
    'California': ('KSFO', (37.6188, -122.375)),
    'Colorado': ('KDEN', (39.8617, -104.6731)),
    'Connecticut': ('KBDL', (41.9389, -72.6832)),
    'Delaware': ('KILG', (39.6787, -75.6065)),
    'Florida': ('KMIA', (25.7933, -80.2906)),
    'Georgia': ('KATL', (33.6367, -84.4281)),
    'Idaho': ('KBOI', (43.5644, -116.2228)),
    'Illinois': ('KORD', (41.9742, -87.9073)),
    'Indiana': ('KIND', (39.7173, -86.2944)),
    'Iowa': ('KDSM', (41.534, -93.6631)),
    'Kansas': ('KICT', (37.6499, -97.4331)),
    'Kentucky': ('KSDF', (38.1744, -85.736)),
    'Louisiana': ('KMSY', (29.9934, -90.258)),
    'Maine': ('KPWM', (43.6462, -70.3093)),
    'Maryland': ('KBWI', (39.1754, -76.6684)),
    'Massachusetts': ('KBOS', (42.3656, -71.0096)),
    'Michigan': ('KDTW', (42.2124, -83.3534)),
    'Minnesota': ('KMSP', (44.882, -93.2218)),
    'Mississippi': ('KJAN', (32.3112, -90.0759)),
    'Missouri': ('KSTL', (38.7487, -90.370)),
    'Montana': ('KBIL', (45.8077, -108.5429)),
    'Nebraska': ('KOMA', (41.3032, -95.8941)),
    'Nevada': ('KLAS', (36.0801, -115.1522)),
    'New Hampshire': ('KMHT', (42.9326, -71.4357)),
    'New Jersey': ('KEWR', (40.6925, -74.1687)),
    'New Mexico': ('KABQ', (35.0402, -106.609)),
    'New York': ('KJFK', (40.6413, -73.7781)),
    'North Carolina': ('KCLT', (35.214, -80.9431)),
    'North Dakota': ('KBIS', (46.7727, -100.746)),
    'Ohio': ('KCMH', (39.998, -82.8919)),
    'Oklahoma': ('KOKC', (35.3931, -97.6007)),
    'Oregon': ('KPDX', (45.5887, -122.5975)),
    'Pennsylvania': ('KPHL', (39.8744, -75.2424)),
    'Rhode Island': ('KPVD', (41.7239, -71.4280)),
    'South Carolina': ('KCHS', (32.8986, -80.0405)),
    'South Dakota': ('KFSD', (43.5820, -96.7419)),
    'Tennessee': ('KBNA', (36.1317, -86.6689)),
    'Texas': ('KDFW', (32.8968, -97.038)),
    'Utah': ('KSLC', (40.7884, -111.9778)),
    'Vermont': ('KBTV', (44.4719, -73.1533)),
    'Virginia': ('KRIC', (37.5052, -77.3197)),
    'Washington': ('KSEA', (47.4489, -122.3094)),
    'West Virginia': ('KCRW', (38.3731, -81.5932)),
    'Wisconsin': ('KMKE', (42.9472, -87.8966)),
    'Wyoming': ('KCYS', (41.1557, -104.811)),
}

# Function to calculate the distance between two coordinates (Haversine formula)
def haversine(coord1, coord2):
    R = 3440.065  # Radius of Earth in nautical miles
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = (math.sin(delta_phi / 2.0) ** 2 +
         math.cos(phi1) * math.cos(phi2) *
         math.sin(delta_lambda / 2.0) ** 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# Create a complete graph where nodes are states and edges are distances between airports
G = nx.complete_graph(len(states_airports))
mapping = {i: state for i, state in enumerate(states_airports.keys())}
G = nx.relabel_nodes(G, mapping)

# Add edge weights (distances with penalties)
for u, v in G.edges():
    coord_u = states_airports[u][1]
    coord_v = states_airports[v][1]
    distance = haversine(coord_u, coord_v)

    # Penalty terms
    penalty = 0
    if distance < 250:
        penalty += (250 - distance) * 2  # Penalize shorter distances
    elif distance > 600:
        penalty += (distance - 600) * 2  # Penalize longer distances

    total_cost = distance + penalty
    G.edges[u, v]['weight'] = total_cost

# Find the approximate shortest path visiting all nodes using the TSP solver from networkx
tsp_path = nx.approximation.traveling_salesman_problem(G, cycle=False, weight='weight')

# Prepare the flight plan
flight_plan = []
for i in range(len(tsp_path) - 1):
    state_from = tsp_path[i]
    state_to = tsp_path[i + 1]
    airport_from = states_airports[state_from][0]
    airport_to = states_airports[state_to][0]
    coord_from = states_airports[state_from][1]
    coord_to = states_airports[state_to][1]
    distance = haversine(coord_from, coord_to)
    flight_plan.append({
        'From State': state_from,
        'From ICAO': airport_from,
        'To State': state_to,
        'To ICAO': airport_to,
        'Distance (nm)': round(distance, 1)
    })

# Display the flight plan
for idx, leg in enumerate(flight_plan, 1):
    print(f"Leg {idx}:")
    print(f"From: {leg['From ICAO']} ({leg['From State']})")
    print(f"To: {leg['To ICAO']} ({leg['To State']})")
    print(f"Distance: {leg['Distance (nm)']} nm\n")


Leg 1:
From: KLAS (Nevada)
To: KSFO (California)
Distance: 359.0 nm

Leg 2:
From: KSFO (California)
To: KPHX (Arizona)
Distance: 564.8 nm

Leg 3:
From: KPHX (Arizona)
To: KABQ (New Mexico)
Distance: 284.9 nm

Leg 4:
From: KABQ (New Mexico)
To: KDEN (Colorado)
Distance: 303.8 nm

Leg 5:
From: KDEN (Colorado)
To: KICT (Kansas)
Distance: 363.9 nm

Leg 6:
From: KICT (Kansas)
To: KOMA (Nebraska)
Distance: 230.6 nm

Leg 7:
From: KOMA (Nebraska)
To: KMSP (Minnesota)
Distance: 244.7 nm

Leg 8:
From: KMSP (Minnesota)
To: KMKE (Wisconsin)
Distance: 257.9 nm

Leg 9:
From: KMKE (Wisconsin)
To: KATL (Georgia)
Distance: 582.3 nm

Leg 10:
From: KATL (Georgia)
To: KCHS (South Carolina)
Distance: 224.7 nm

Leg 11:
From: KCHS (South Carolina)
To: KMIA (Florida)
Distance: 426.8 nm

Leg 12:
From: KMIA (Florida)
To: KMSY (Louisiana)
Distance: 585.6 nm

Leg 13:
From: KMSY (Louisiana)
To: KBHM (Alabama)
Distance: 279.1 nm

Leg 14:
From: KBHM (Alabama)
To: KLIT (Arkansas)
Distance: 280.7 nm

Leg 15:
From: KLI